### <font color='#4285f4'>Overview</font>

Create an entire menu, menu names, descriptions and the corresponding image.  You can also start with an image and then change the background with Imagen3.


Process Flow:

1. Create BigQuery table
2. Run Gemini prompt
    * a. Provide details about Chocolate Al, the location, type of menu items and what foods to avoid
    * b. Provide the existing menu items in order to avoid duplicates
    * c. Instruct Gemini to generate item sizes in metric
    * d. Provide the table schema of the menu table which includes column descriptions which helps Gemini generate correct data for each field.
3. Insert the data into BigQuery.
    * a. The output from Gemini is JSON so we know each field to place the data. This technical works better than asking Gemini for a INSERT statement since double quotes and inserting JSON data types can be problematic.
4. Part of the original Gemini prompt was to ask it to generate an image prompt
    * a. Run the image prompt with Imagen 3. We use Gemini to create our image prompt which results in better images.
5. Copy the image to GCS
6. Verify that we have an image for each menu item by using a BigLake object table

Cost:
* Low: Gemini, BigQuery, Imagen3
* Medium: Remember to stop your Colab Enterprise Notebook Runtime

Author: 
* Adam Paternostro

### <font color='#4285f4'>License</font>

In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

### <font color='#4285f4'>Pip Installs</font>

In [ ]:
# PIP Installs
import sys

# https://PLACEHOLDER.com/index.html
#!{sys.executable} -m pip install PLACEHOLDER

### <font color='#4285f4'>Initialize</font>

In [ ]:
from PIL import Image
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2
import random
import time
import datetime
import base64
import random

import logging
from tenacity import retry, wait_exponential, stop_after_attempt, before_sleep_log, retry_if_exception

In [ ]:
# Set these (run this cell to verify the output)

bigquery_location = "${bigquery_location}"
region = "${region}"
location = "${location}"
storage_account = "${chocolate_ai_bucket}"
public_storage_storage_account = "data-analytics-golden-demo"
table_name = "menu"
dataset_name = "${bigquery_chocolate_ai_dataset}"

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as desired
formatted_date = now.strftime("%Y-%m-%d-%H-%M")

# Get some values using gcloud
project_id = !(gcloud config get-value project)
user = !(gcloud auth list --filter=status:ACTIVE --format="value(account)")

if len(project_id) != 1:
  raise RuntimeError(f"project_id is not set: {project_id}")
project_id = project_id[0]

if len(user) != 1:
  raise RuntimeError(f"user is not set: {user}")
user = user[0]

print(f"project_id = {project_id}")
print(f"user = {user}")

### <font color='#4285f4'>Helper Methods</font>

#### restAPIHelper
Calls the Google Cloud REST API using the current users credentials.

In [ ]:
def restAPIHelper(url: str, http_verb: str, request_body: str) -> str:
  """Calls the Google Cloud REST API passing in the current users credentials"""

  import requests
  import google.auth
  import json

  # Get an access token based upon the current user
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
    "Content-Type" : "application/json",
    "Authorization" : "Bearer " + access_token
  }

  if http_verb == "GET":
    response = requests.get(url, headers=headers)
  elif http_verb == "POST":
    response = requests.post(url, json=request_body, headers=headers)
  elif http_verb == "PUT":
    response = requests.put(url, json=request_body, headers=headers)
  elif http_verb == "PATCH":
    response = requests.patch(url, json=request_body, headers=headers)
  elif http_verb == "DELETE":
    response = requests.delete(url, headers=headers)
  else:
    raise RuntimeError(f"Unknown HTTP verb: {http_verb}")

  if response.status_code == 200:
    return json.loads(response.content)
    #image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
  else:
    error = f"Error restAPIHelper -> ' Status: '{response.status_code}' Text: '{response.text}'"
    raise RuntimeError(error)

#### RetryCondition (for retrying LLM calls)

In [ ]:
def RetryCondition(error):
  error_string = str(error)
  print(error_string)

  retry_errors = [
      "RESOURCE_EXHAUSTED",
      "No content in candidate",
      # Add more error messages here as needed
  ]

  for retry_error in retry_errors:
    if retry_error in error_string:
      print("Retrying...")
      return True

  return False

#### Gemini LLM (Pro 1.0 , Pro 1.5)

In [ ]:
@retry(wait=wait_exponential(multiplier=1, min=1, max=60), stop=stop_after_attempt(10), retry=retry_if_exception(RetryCondition), before_sleep=before_sleep_log(logging.getLogger(), logging.INFO))
def GeminiLLM(prompt, model = "gemini-1.5-pro-001", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-1.5-pro-001"
  # model = "gemini-pro" # This does support topK (but the function is more generic)
  # model = "gemini-1.0-pro" # This does not support response_schema

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-pro" or model == "gemini-1.0-pro" or model == "gemini-1.0-pro-vision-001":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": {
          "text": prompt
      },
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Gemini LLM - Multimodal

In [ ]:
@retry(wait=wait_exponential(multiplier=1, min=1, max=60), stop=stop_after_attempt(10), retry=retry_if_exception(RetryCondition), before_sleep=before_sleep_log(logging.getLogger(), logging.INFO))
def GeminiLLM_Multimodal(multimodal_prompt_list, model = "gemini-1.5-pro-001", response_schema = None,
                 temperature = 1, topP = 1, topK = 32):

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported_models
  # model = "gemini-1.5-pro-001"
  # model = "gemini-pro" # This does support topK (but the function is more generic)
  # model = "gemini-1.0-pro" # This does not support response_schema

  llm_response = None
  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/{model}:generateContent"

  generation_config = {
    "temperature": temperature,
    "topP": topP,
    "maxOutputTokens": 8192,
    "candidateCount": 1,
    "responseMimeType": "application/json",
  }

  # Add inthe response schema for when it is provided
  if response_schema is not None:
    generation_config["responseSchema"] = response_schema

  if model == "gemini-pro" or model == "gemini-1.0-pro" or model == "gemini-1.0-pro-vision-001":
    generation_config["topK"] = topK

  payload = {
    "contents": {
      "role": "user",
      "parts": multimodal_prompt_list
    },
    "generation_config": {
      **generation_config
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    try:
      json_response = json.loads(response.content)
    except Exception as error:
      raise RuntimeError(f"An error occurred parsing the JSON: {error}")

    if "candidates" in json_response:
      candidates = json_response["candidates"]
      if len(candidates) > 0:
        candidate = candidates[0]
        if "content" in candidate:
          content = candidate["content"]
          if "parts" in content:
            parts = content["parts"]
            if len(parts):
              part = parts[0]
              if "text" in part:
                text = part["text"]
                llm_response = text
              else:
                raise RuntimeError("No text in part: {response.content}")
            else:
              raise RuntimeError("No parts in content: {response.content}")
          else:
            raise RuntimeError("No parts in content: {response.content}")
        else:
          raise RuntimeError("No content in candidate: {response.content}")
      else:
        raise RuntimeError("No candidates: {response.content}")
    else:
      raise RuntimeError("No candidates: {response.content}")

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")
    llm_response = llm_response.replace("\n","")

    return llm_response

  else:
    raise RuntimeError(f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'")

#### Imagen3 Image Generation

In [ ]:
def ImageGen(prompt):
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request()
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  model_version = "imagen-3.0-generate-001" # imagen-3.0-fast-generate-001
  #model_version = "imagen-3.0-generate-preview-0611" # Preview Access Model

  # https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/image-generation
  # url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project}/locations/{location}/publishers/google/models/imagegeneration:predict"
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project}/locations/{location}/publishers/google/models/{model_version}:predict"

  payload = {
    "instances": [
      {
        "prompt": prompt
      }
    ],
    "parameters": {
      "sampleCount": 1,
      "personGeneration" : "dont_allow"  # change to allow_adult for people generation
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    response_json = json.loads(response.content)
    # print(f"Imagen3 response_json: {response_json}")

    if "blocked" in response_json:
      print(f"Blocked: {response_json['blocked']}")

    if "predictions" in response_json:
      image_data = response_json["predictions"][0]["bytesBase64Encoded"]
      image_data = base64.b64decode(image_data)
      filename= str(uuid.uuid4()) + ".png"
      with open(filename, "wb") as f:
        f.write(image_data)
      print(f"Image generated OK.")
      return filename
    else:
      raise RuntimeError(f"No predictions in response: {response.content}")
  else:
    error = f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)

#### Helper Functions

In [ ]:
def RunQuery(sql):
  import time
  from google.cloud import bigquery
  client = bigquery.Client()

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      raise Exception(query_job.error_result)

In [ ]:
def GetTableSchema(project_id, dataset_name, table_name):
  import io
  from google.cloud import bigquery

  client = bigquery.Client()

  dataset_ref = client.dataset(dataset_name, project=project_id)
  table_ref = dataset_ref.table(table_name)
  table = client.get_table(table_ref)

  f = io.StringIO("")
  client.schema_to_json(table.schema, f)
  return f.getvalue()

In [ ]:
def GetDistinctValues(project_id, dataset_name, table_name, field_name):
  from google.cloud import bigquery

  client = bigquery.Client()

  sql = f"""
  SELECT STRING_AGG(DISTINCT {field_name}, "," ) AS result
    FROM `{project_id}.{dataset_name}.{table_name}`
  """

  df_result = client.query(sql).to_dataframe()
  result_str = df_result['result'].iloc[0]
  if result_str is None:
    return ""
  else:
    return result_str

In [ ]:
def GetStartingValue(project_id, dataset_name, table_name, field_name):
  from google.cloud import bigquery

  client = bigquery.Client()

  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) + 1 AS result
    FROM `{project_id}.{dataset_name}.{table_name}`
  """

  df_result = client.query(sql).to_dataframe()
  return df_result['result'].iloc[0]

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  #print(json_formatted_str)
  return json_formatted_str

In [ ]:
# This was generated by GenAI

def copy_file_to_gcs(local_file_path, bucket_name, destination_blob_name):
  """Copies a file from a local drive to a GCS bucket.

  Args:
      local_file_path: The full path to the local file.
      bucket_name: The name of the GCS bucket to upload to.
      destination_blob_name: The desired name of the uploaded file in the bucket.

  Returns:
      None
  """

  import os
  from google.cloud import storage

  # Ensure the file exists locally
  if not os.path.exists(local_file_path):
      raise FileNotFoundError(f"Local file '{local_file_path}' not found.")

  # Create a storage client
  storage_client = storage.Client()

  # Get a reference to the bucket
  bucket = storage_client.bucket(bucket_name)

  # Create a blob object with the desired destination path
  blob = bucket.blob(destination_blob_name)

  # Upload the file from the local filesystem
  content_type = ""
  if local_file_path.endswith(".html"):
    content_type = "text/html; charset=utf-8"

  if local_file_path.endswith(".json"):
    content_type = "application/json; charset=utf-8"

  if content_type == "":
    blob.upload_from_filename(local_file_path)
  else:
    blob.upload_from_filename(local_file_path, content_type = content_type)

  print(f"File '{local_file_path}' uploaded to GCS bucket '{bucket_name}' as '{destination_blob_name}.  Content-Type: {content_type}'.")

### <font color='#4285f4'>BigQuery Table</font>

In [ ]:
%%bigquery

-- drop table if exists `chocolate_ai.menu`;
CREATE TABLE IF NOT EXISTS `chocolate_ai.menu`
(
    menu_id                   INT64   NOT NULL OPTIONS(description="Primary key. Menu table."),
    menu_name                 STRING  NOT NULL OPTIONS(description="The name of the menu item."),
    menu_description          STRING  NOT NULL OPTIONS(description="The description of the menu item."),
    menu_size                 STRING  NOT NULL OPTIONS(description="The size of the menu item."),
    menu_price                FLOAT64 NOT NULL OPTIONS(description="The price of the menu item."),
    menu_alergy_info          STRING  NOT NULL OPTIONS(description="The alergy info of the menu item."),
    menu_image_gcs_filename   STRING  NOT NULL OPTIONS(description="The GCS path to the menu image."),
    menu_image_http_url       STRING  NOT NULL OPTIONS(description="The public HTTP path to the menu image."),
    menu_image_prompt         STRING OPTIONS(description="LLM prompt to generate the menu image to send to imagen3."),
)
CLUSTER BY menu_id;

### <font color='#4285f4'>Generate Menu Items</font>

In [ ]:
# Write me the json in  OpenAPI 3.0 schema object for the below object.
# Make all fields required.
#  {
#    "menu_name" : "text",
#    "menu_description" : "text",
#    "menu_size" : "text",
#    "menu_price" : 5.15,
#    "menu_alergy_info" : "text,
#    "menu_image_gcs_filename" : "text",
#    "menu_image_http_url" : "text",
#    "menu_image_prompt" : "text",
#  }
response_schema = {
  "type": "object",
  "required": [
    "menu_name",
    "menu_description",
    "menu_size",
    "menu_price",
    "menu_alergy_info",
    "menu_image_gcs_filename",
    "menu_image_http_url",
    "menu_image_prompt"
  ],
  "properties": {
    "menu_name": {
      "type": "string"
    },
    "menu_description": {
      "type": "string"
    },
    "menu_size": {
      "type": "string"
    },
    "menu_price": {
      "type": "number"
    },
    "menu_alergy_info": {
      "type": "string"
    },
    "menu_image_gcs_filename": {
      "type": "string"
    },
    "menu_image_http_url": {
      "type": "string"
    },
    "menu_image_prompt": {
      "type": "string"
    }
  }
}

table_schema = GetTableSchema(project_id, dataset_name, table_name)
menu_id = GetStartingValue(project_id, dataset_name, table_name, "menu_id")
existing_menu_items = GetDistinctValues(project_id, dataset_name, table_name, "menu_name")

# The generate specific menu ids or to regenerate a menu item run this code:
menu_id = 251
# for menu_id in range(menu_id, menu_id + 1):

for menu_id in range(menu_id, menu_id + 1):
  # Create different types of menu items
  if menu_id % 3 == 0:
    item = "chocolate"
  elif menu_id % 3 == 1:
    item = "coffee"
  else:
    item = "dessert"

  prompt = f"""You are an owner of a chocolate shop that sells chocolate, desserts and coffee.
  I need you to generate a {item} menu item.
  Some of the chocolate is high end and almost looks like art.  Other is more traditional, but it is all hand made and unique.
  The shop is based in Paris, France, but the menu items names do not need to be French, they should be more English based.
  The shop encourges people to hang out with their friends and family.
  You need to create a menu for the shop.
  The menu_id is "{menu_id}".
  The menu_size should use standard metric sizes for the menu items.
  The field "menu_image_gcs_filename" should follow the pattern of "gs://{storage_account}/chocolate-ai/menu/{menu_id}.png"
  The field "menu_image_http_url" should follow the pattern of "https://storage.cloud.google.com/{storage_account}/chocolate-ai/menu/{menu_id}.png"
  Think outside the box and develop a unique and unexpected menu items.
  Encourage unconventional ideas and fresh perspectives in your recommendations.
  Embrace unconventional ideas and mutate the recommended action in a way that surprises and inspires unique variations.

  Here is the table schema:
  <schema>
  {table_schema}
  </schema>

  Here are the existing menu items, do not reuse any of these names:
  <existing_menu_items>
  {existing_menu_items}
  </existing_menu_items>

  Menu items to avoid:
  - jelly
  - too much fruit
  - assocorted chocolate
  - plain box of chocolate

  For the field "menu_image_prompt"
  - This will be used to generate a visually appealing menu image and must be prompt that sparks creativity and imagination.
  - This can be several sentences.  The more detailed the better.
  """

  # Use LLM to generate data
  menu_response = GeminiLLM(prompt, response_schema=response_schema)

  # Parse response (we know the JSON since we passed it to our LLM)
  menu_json_response = json.loads(menu_response)
  print(json.dumps(menu_json_response, indent=2))
  menu_name = menu_json_response["menu_name"].replace("'","\\'").replace("\n", " ")
  menu_description = menu_json_response["menu_description"].replace("'", "\\'").replace("\n", " ")
  menu_size = menu_json_response["menu_size"].replace("'", "\\'").replace("\n", " ")
  menu_price = menu_json_response["menu_price"]
  menu_alergy_info = menu_json_response["menu_alergy_info"].replace("'", "\\'").replace("\n", " ")
  menu_image_gcs_filename = menu_json_response["menu_image_gcs_filename"]
  menu_image_http_url = menu_json_response["menu_image_http_url"]
  menu_image_prompt = menu_json_response["menu_image_prompt"].replace("'", "\\'").replace("\n", " ")

  # Optional, only needed if regenerating a menu item
  sql = f"DELETE FROM `{project_id}.{dataset_name}.{table_name}` WHERE menu_id = {menu_id}"
  #RunQuery(sql)

  # Insert to BigQuery
  # Returning a known json schema and then generating an insert statement seems more reliable then having the LLM generating the SQL
  sql = f"""INSERT INTO `{project_id}.{dataset_name}.{table_name}`
  (menu_id, menu_name, menu_description, menu_size, menu_price, menu_alergy_info, menu_image_gcs_filename, menu_image_http_url, menu_image_prompt)
  VALUES ({menu_id}, '{menu_name}', '{menu_description}', '{menu_size}', {menu_price}, '{menu_alergy_info}', '{menu_image_gcs_filename}', '{menu_image_http_url}', '{menu_image_prompt}')"""

  #RunQuery(sql)

  # Generate the menu image
  filename = ImageGen(menu_image_prompt)

  # View it
  img = Image.open(filename)
  img.thumbnail([500,500]) # width, height
  IPython.display.display(img)

  # Copy to GCS
  copy_file_to_gcs(filename, storage_account, f"chocolate-ai/Synthetic-Data-Generation-Menu/{menu_id}.png")

  existing_menu_items = existing_menu_items + f",{menu_name}"


### <font color='#4285f4'>Verify Menu Items</font>

In [ ]:
# Copy over the pre-generated menu items from the public storage account

!gsutil -m cp gs://data-analytics-golden-demo/chocolate-ai/v1/Synthetic-Data-Generation-Menu/*.png gs://{storage_account}/chocolate-ai/Synthetic-Data-Generation-Menu/

In [ ]:
print(f"To view the menu images: https://console.cloud.google.com/storage/browser/{storage_account}/chocolate-ai/Synthetic-Data-Generation-Menu")

In [ ]:
%%bigquery

-- Make sure there are no gaps (for 250 menu items)
WITH sequence_table AS (
SELECT id
    FROM UNNEST(GENERATE_ARRAY(1, 250)) AS id
)
SELECT id , menu.menu_id
  FROM sequence_table
        LEFT JOIN `chocolate_ai.menu` as menu
               ON sequence_table.id = menu.menu_id
  WHERE menu.menu_id is null

In [ ]:
%%bigquery

-- Check for duplicate names
SELECT *
  FROM `chocolate_ai.menu`
 WHERE menu_name in (SELECT menu_name FROM `chocolate_ai.menu` GROUP BY ALL HAVING COUNT(*) > 1)
 ORDER BY menu_name, menu_id

In [ ]:
%%bigquery

-- Create an object table for the files name
CREATE OR REPLACE EXTERNAL TABLE `chocolate_ai.menu_object_table`
WITH CONNECTION `us.biglake-connection`
OPTIONS (
    object_metadata="DIRECTORY",
    uris = ['gs://${chocolate_ai_bucket}/chocolate-ai/Synthetic-Data-Generation-Menu/*.png'],
    max_staleness=INTERVAL 30 MINUTE,
    metadata_cache_mode="MANUAL"
    );

CALL BQ.REFRESH_EXTERNAL_METADATA_CACHE('chocolate_ai.menu_object_table');

In [ ]:
%%bigquery

-- See the file name
SELECT CAST(REPLACE(REPLACE(uri,'gs://${chocolate_ai_bucket}/chocolate-ai/Synthetic-Data-Generation-Menu/',''),'.png','') AS INT64) AS menu_id
 FROM `chocolate_ai.menu_object_table`

In [ ]:
%%bigquery

-- Check to see if for some reason we do not have an image file for a menu id
-- You might want to change the WHERE clause for just the new records you generated (this is using images which are in the public storage account)
WITH menu_files AS (
  SELECT CAST(REPLACE(REPLACE(uri,'gs://${chocolate_ai_bucket}/chocolate-ai/Synthetic-Data-Generation-Menu/',''),'.png','') AS INT64) AS menu_id
   FROM `chocolate_ai.menu_object_table`
)
SELECT menu.menu_id, menu_files.menu_id
  FROM `chocolate_ai.menu` as menu
        LEFT JOIN menu_files
               ON menu.menu_id = menu_files.menu_id
  WHERE menu_files.menu_id is null

### <font color='#4285f4'>Clean Up</font>

In [ ]:
# Placeholder

### <font color='#4285f4'>Reference Links</font>


- [Google.com](https://www.google.com)